In [1]:
import zipfile
import databricks.koalas as ks
import sys
sys.path.insert(0, '../')
from util import util_data_access
from pyspark.sql import SparkSession
import tqdm
import pandas as pd 
import os, glob
import re
from tqdm.auto import tqdm

In [2]:
from importlib import reload

# Download and Prepare Data

In [14]:
## download spacy model 
# download data
util_data_access.download_file('en_core_web_lg.tar.gz', 'edit-pathways/spacy/en_core_web_lg.tar.gz')
! tar -xf en_core_web_lg.tar.gz
! mv en_core_web_lg-2.3.1 en_core_web_lg

True

In [3]:
# download data
util_data_access.download_file('newssniffer-nytimes.db.gz', 'edit-pathways/dbs/newssniffer-nytimes.db.gz')
! gunzip newssniffer-nytimes.db.gz

True

In [7]:
## download data
# util_data_access.download_file('edit-data.zip', 'edit-data/edit-data.zip')
# myzip = zipfile.ZipFile('edit-data.zip')
# myzip.extractall(path='edit-data')

True

In [68]:
util_data_access.upload_file(
    'edit-data/article_output/article-output-1-8.json',
    object_name='edit-data/article-output-1-8.json',
)

True

In [71]:
for i in tqdm(range(2, 9)):
    util_data_access.upload_file(
        'edit-data/article_output/article-output-%d-8.json' % i,
        object_name='edit-data/article-output-%d-8.json' % i,
    )

In [17]:
for i in tqdm(range(1, 9)):
    util_data_access.upload_file(
        'edit-data/article_output/version-output-%d-8.json' % i,
        object_name='edit-data/version-output-%d-8.json' % i,
    )

# Turn on Spark

In [22]:
# turn on spark
spark = (
    SparkSession.builder \
      .config("spark.executor.instances", "50") \
      .config("spark.driver.memory", "15g") \
      .getOrCreate()
)
# sc = spark.sparkContext
# set arrow to true
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
spark.conf.set("spark.sql.execution.arrow.enabled", "false")
ks.set_option('compute.default_index_type', 'distributed')
ks.set_option('compute.shortcut_limit',1) # Koalas will apply pyspark 

In [24]:
spark

In [5]:
import sqlite3
import sys
sys.path.append('../util')
import util_newssniffer_parsing as unp
reload(unp)

In [6]:
conn = sqlite3.connect('newssniffer-nytimes.db')

In [7]:
df = pd.read_sql('select * from entryversion where entry_id in (select distinct entry_id from entryversion limit 200)', con=conn)

In [25]:
kdf = ks.from_pandas(df)

In [26]:
tdf = kdf.to_pandas()

In [29]:
grouped_dfs = (kdf
               .groupby('entry_id')
               .apply(lambda df: unp.get_sentence_diff_stats_on_article(df, get_sentence_vars=True, ret_type='list',))
              )

ArrowInvalid: Could not convert ([{'num_added_sents': 0, 'len_new_doc': 64, 'num_removed_sents': 0, 'len_old_doc': 64, 'num_changed_sents': 0, 'version_nums': (0, 1), 'a_id': 547988, 'vars_old': ({'text': 'In Silicon Valley, Apple just won big against Samsung in the patent lawsuit of the year, after trading claims and counterclaims of pilfered product ideas.', 'tag': ' '}, {'text': 'Across the country, in a federal court in Florida, an inventor named Mark Stadnyk is waging a different kind of patent warfare — an ambitious and perhaps quixotic legal foray.', 'tag': ' '}, {'text': 'Mr. Stadnyk, who holds a patent on a motorcycle windshield, is suing the United States government, aiming to head off a patent law that he says will favor big companies and hurt lone inventors like himself.', 'tag': ' '}, {'text': 'Represented by a prominent Washington lawyer, Mr. Stadnyk filed a suit last month that challenges the constitutionality of legislation that Congress passed last fall, the America In

In [14]:
sample_df = df.loc[lambda df: df['entry_id'].isin(df['entry_id'].drop_duplicates().sample(10))]

In [16]:
grouped_dfs = (sample_df
               .groupby('entry_id')
               .apply(lambda df: unp.get_sentence_diff_stats_on_article(a=df, get_sentence_vars=True))
              )

In [30]:
sample_df.shape

(1995, 12)

In [20]:
grouped_dfs.iloc[0]

(     a_id  len_new_doc  len_old_doc  num_added_sents  num_changed_sents  \
 0  588653           41           41                0                  3   
 
    num_removed_sents                                           vars_new  \
 0                  0  ({'text': 'WASHINGTON—President Obama is leani...   
 
                                             vars_old version_nums  
 0  ({'text': 'WASHINGTON—President Obama is leani...       (1, 2)  ,
      a_id  len_new_sent  len_old_sent  num_added_words  num_removed_words  \
 0  588653            27            16               16                  5   
 
    s_idx                                              s_new  \
 0     40  [{'text': '', 'tag': ' '}, {'text': 'Mr.', 'ta...   
 
                                                s_old version_nums  
 0  [{'text': 'He', 'tag': '-'}, {'text': '', 'tag...       (1, 2)  )

# Data Manipulation

In [5]:
cache_dir = 'edit-data/article_output/'

article_page_files = glob.glob(cache_dir + '/article-output*')
diff_files         = glob.glob(cache_dir + '/diff-output-*')
version_files      = glob.glob(cache_dir + '/version-output*')

sort_by_num = lambda fs: sorted(fs, key=lambda x: int(re.search('-(\d+)-\d.json', x)[1]))
article_page_files = sort_by_num(article_page_files)
diff_files = sort_by_num(diff_files)
version_files = sort_by_num(version_files)

In [6]:
article_kdfs = None
for i in tqdm(range(1, 9)):
    kdf = ks.read_json('s3://aspangher/edit-data/article-output-%d-8.json' % i)#, index_col=0)
    if article_kdfs is None:
        article_kdfs = kdf
    else:
        article_kdfs = article_kdfs.append(kdf, ignore_index=True)

In [9]:
article_kdfs.shape 

(2791957, 2)

In [7]:
article_kdfs = article_kdfs.drop('_corrupt_record').dropna()

In [8]:
version_files = sort_by_num(version_files)

In [9]:
version_kdfs = None
for i in tqdm(range(1, 9)):
    kdf = ks.read_json('s3://aspangher/edit-data/version-output-%d-8.json' % i)#, index_col=0)
    if version_kdfs is None:
        version_kdfs = kdf
    else:
        version_kdfs = version_kdfs.append(kdf, ignore_index=True)

In [10]:
version_kdfs.shape 

(3470064, 9)

In [11]:
version_kdfs = version_kdfs.drop('_corrupt_record').dropna()

In [12]:
version_kdfs.columns

Index(['article_id', 'article_url', 'source', 'time_created', 'time_delta',
       'title', 'version', 'version_url'],
      dtype='object')

In [13]:
version_kdfs.head(1)

,article_id,article_url,source,time_created,time_delta,title,version,version_url
1,35,http://news.bbc.co.uk/go/rss/-/1/hi/world/midd...,bbc,"August 29, 2006 01:35",,Annan asks Israel to end blockade,0,https://www.newssniffer.co.uk/articles/35/vers...


In [14]:
article_kdfs.columns

Index(['html', 'url'], dtype='object')

In [29]:
full_kdfs = version_kdfs.merge(article_kdfs, left_on='version_url', right_on='url')

In [16]:
full_kdfs.head(1)

,article_id,article_url,source,time_created,time_delta,title,version,version_url,html,url
0,10001,http://news.bbc.co.uk/go/rss/-/1/hi/uk/6115912...,bbc,"November 05, 2006 21:20",1 day later,North East 'warms to devolution',2,https://www.newssniffer.co.uk/articles/10001/v...,"<p><a href=""http://news.bbc.co.uk/go/rss/-/1/h...",https://www.newssniffer.co.uk/articles/10001/v...


In [18]:
t = full_kdfs['html'].head(2)

In [21]:
s = t.iloc[0]

In [30]:
def f(s):
    return '</p><p>'.join(s.split('</p><p>')[2:])

full_kdfs['body_text'] = full_kdfs['html'].apply(f)

In [33]:
t = full_kdfs['body_text'].str.split('</p><p>')

In [38]:
from collections import defaultdict, Counter

In [39]:
c = Counter()

In [ ]:
counter = defaultdict(int)
for i in tqdm(t):
    c.update(i)

In [49]:
full_kdfs.columns

Index(['article_id', 'article_url', 'source', 'time_created', 'time_delta',
       'title', 'version', 'version_url', 'html', 'url', 'body_text'],
      dtype='object')

In [50]:
t = full_kdfs.groupby('article_id')[['version', 'body_text', 'title']]

In [52]:
t.head(1)

Py4JJavaError: An error occurred while calling o3814.collectToPython.
: org.apache.spark.SparkException: Job 32 cancelled 
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1889)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1824)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:2076)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2059)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2048)
	at org.apache.spark.util.EventLoop$$anon$1.run(EventLoop.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.runJob(DAGScheduler.scala:737)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2061)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2082)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2101)
	at org.apache.spark

KeyboardInterrupt: 

In [31]:
ks.Series

databricks.koalas.series.Series

In [34]:
full_kdfs['version_url'].isnull().value_counts()

False    3470048
Name: version_url, dtype: int64

In [ ]:
spark.stop()